In [ ]:
# ==============================================================================
# CE NOTEBOOKE EST LA MISE AU PROPRE DU NOTEBOOK :
#
# 4.1-fh-modeling-advanced-text.ipynb & 3.0-fdm-deep-et-machine-learning-texte.ipynb
#
# ==============================================================================

**⚠️ Avertissement sur l'Exécution et l'Origine des Fichiers**

En raison de la charge de calcul intensive requise pour l'entraînement des modèles de Deep Learning (tels que CamemBERT et Flaubert), le code de ce notebook n'a pas été ré-exécuté dans cet environnement.

Par conséquent, les fichiers de sortie générés par ce notebook (principalement les fichiers de **logits** comme `camembert_logits.pt`, `flaubert_logits.pt`, etc.) ont été créés au préalable lors d'une exécution du notebook d'origine sur un serveur de calcul plus puissant et payant.

PARTIE 1 : Entraînement des Modèles de Base (CamemBERT & Flaubert)

In [ ]:
# ==============================================================================
# NOTEBOOK D'ENTRAÎNEMENT DE MODÈLES DE CLASSIFICATION DE TEXTE
#
# Objectif : Entraîner, évaluer et sauvegarder deux modèles de NLP (CamemBERT
# et Flaubert) sur une tâche de classification.
#
# ==============================================================================

In [ ]:
!pip install sacremoses -q

In [ ]:
# Montage de Google Drive pour accéder aux fichiers depuis Colab
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# ====== 1. BIBLIOTHEQUES ======

# Outils système et gestion de fichiers
import os
import json
import shutil
import gc
import warnings

# Manipulation de données
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# PyTorch
import torch
from torch.nn import CrossEntropyLoss
from torch.utils.data import Dataset, Subset

# Hugging Face Transformers
from transformers import (
    CamembertTokenizer,
    CamembertForSequenceClassification,
    FlaubertTokenizer,
    FlaubertForSequenceClassification,
    FlaubertConfig,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
    EarlyStoppingCallback,
    TrainerCallback,
    set_seed,
)

# Scikit-learn
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
# ====== 2. PARAMÉTRAGE DE L'ENVIRONNEMENT ======

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
warnings.filterwarnings("ignore")

print(f"Utilisation de l'appareil : {device}")

# Définition des chemins
BASE_PATH = "/content/drive/MyDrive/Colab Notebooks/data_rakuten/"
LOCAL_PATH = '/content/data/'

# Création des dossiers
os.makedirs(LOCAL_PATH, exist_ok=True)
os.makedirs(os.path.join(BASE_PATH, "models"), exist_ok=True)

In [ ]:
# AVERTISSEMENT :

# Google Colab fonctionne sur des machines virtuelles temporaires.
# Tous les fichiers enregistrés localement (dans /content/) seront supprimés lorsque :
#   - La session expire,
#   - Le runtime est réinitialisé,
#   - Ou que le notebook est fermé trop longtemps.
#
# Bonnes pratiques :
#   - Conservez toujours vos fichiers originaux dans Google Drive.
#   - Rechargez ou recopiez-les au début de chaque session si vous en avez besoin dans l'environnement local.
#   - Montez Google Drive avec `drive.mount()` puis copiez les fichiers nécessaires dans `/content/`.

# Travailler depuis l’espace local de Colab (/content/) :
# Avantages :
#   - Accès plus rapide aux fichiers (lecture/écriture bien plus performante qu’avec Drive).
#   - Moins de risques d’erreurs de chemin ou de synchronisation.
#   - Plus compatible avec certains outils et bibliothèques.
#
# Inconvénients :
#   - Les fichiers sont supprimés à chaque redémarrage de session
#   - Espace disque limité.
#
# Recommandation :
# Utilisez Google Drive pour le stockage persistant, mais travaillez sur une copie locale dans `/content/` pour de meilleures performances.

In [ ]:
# ====== 3. PRÉPARATION DES DONNÉES EN LOCAL ======

print("--- Copie des données depuis Google Drive vers l'espace local ---")
# Copie du fichier CSV principal pour un accès rapide.
shutil.copy(os.path.join(BASE_PATH, 'X_train_fr_final.csv'), LOCAL_PATH)
# Copie des fichiers annexes
shutil.copy(os.path.join(BASE_PATH, 'label_encoder_final.pkl'), LOCAL_PATH)

print("Fichiers copiés dans l'espace local de Colab :")
print(os.listdir(LOCAL_PATH))

In [ ]:
# ====== 4. CHARGEMENT ET SÉPARATION DES DONNÉES ======

# Chargement des données principales depuis les fichiers locaux
df = pd.read_csv(os.path.join(LOCAL_PATH, "X_train_fr_final.csv"))
le = joblib.load(os.path.join(LOCAL_PATH, 'label_encoder_final.pkl'))
texts = df['texte_nettoye_traduit'].tolist()
labels = df['label'].tolist()

# Définir le chemin du fichier d'indices sur Google Drive
val_indices_path_drive = os.path.join(BASE_PATH, 'val_indices.json')

# Vérifier si les indices de validation existent déjà sur le Drive
if os.path.exists(val_indices_path_drive):
    print("Indices de validation existants trouvés. Chargement...")
    # Charger les indices depuis le Drive
    with open(val_indices_path_drive, 'r') as f:
        val_idx = np.array(json.load(f), dtype=int)
else:
    print("Aucun indice de validation trouvé. Création d'une nouvelle séparation stratifiée...")
    # Créer les indices
    # Nous voulons garder 15% pour la validation (taille du dataset initial : 84916)
    # 13138 / 84916 ≈ 0.1547
    train_full_idx, val_idx = train_test_split(
        np.arange(len(df)),
        test_size=0.1547,
        random_state=42,
        stratify=labels # Assure une répartition équilibrée des classes
    )

    # Sauvegarder les nouveaux indices sur Google Drive pour la prochaine exécution
    with open(val_indices_path_drive, 'w') as f:
        json.dump(val_idx.tolist(), f)
    print(f"Nouveaux indices sauvegardés dans : {val_indices_path_drive}")

# Les indices d'entraînement sont ceux qui ne sont pas dans l'ensemble de validation
train_idx = np.setdiff1d(np.arange(len(df)), val_idx)

print(f"\n Séparation des données terminée :")
print(f"   - {len(train_idx)} échantillons d'entraînement")
print(f"   - {len(val_idx)} échantillons de validation")

In [ ]:
# ====== 5. DÉFINITIONS DES CLASSES ET FONCTIONS UTILES ======

# Cette section centralise toutes les classes et fonctions personnalisées
# nécessaires pour l'entraînement.

class TextDS(Dataset):
    """
    Dataset PyTorch pour charger et tokeniser des données textuelles.

    Cette classe prend une liste de textes et de labels, et utilise un tokenizer
    Hugging Face fourni pour convertir les textes en encodages numériques
    (input_ids, attention_mask). Ces encodages sont directement compatibles
    avec les modèles de la bibliothèque Transformers comme CamemBERT ou Flaubert.

    Attributes:
        encodings (transformers.tokenization_utils_base.BatchEncoding): Les textes tokenizés.
        labels (list): La liste des labels numériques.
    """
    def __init__(self, texts, labels, tokenizer, max_length=512):
        """
        Initialise et tokenise l'ensemble des données textuelles.

        Args:
            texts (list[str]): La liste des chaînes de caractères à traiter.
            labels (list[int]): La liste des labels entiers correspondants.
            tokenizer (transformers.PreTrainedTokenizer): L'instance du tokenizer
                pré-entraîné (ex: CamembertTokenizer) à utiliser.
            max_length (int, optional): La longueur maximale des séquences après
                tokenisation. Les textes plus longs seront tronqués. Défaut à 512.
        """
        self.encodings = tokenizer(texts, padding=True, truncation=True, max_length=max_length, return_tensors="pt")
        self.labels = labels

    def __len__(self):
        """
        Retourne le nombre total d'échantillons dans le dataset.

        Returns:
            int: La taille totale du dataset.
        """
        return len(self.labels)

    def __getitem__(self, i):
        """
        Récupère un échantillon (encodage et label) à un index donné.

        Args:
            i (int): L'index de l'échantillon à retourner.

        Returns:
            dict: Un dictionnaire contenant les encodages (ex: 'input_ids',
                  'attention_mask') et le 'labels' sous forme de tenseurs,
                  prêt à être consommé par le modèle.
        """
        item = {k: v[i] for k, v in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[i], dtype=torch.long)
        return item

class WeightedTrainer(Trainer):
    """
    Subclasse du Trainer de Hugging Face pour utiliser une perte pondérée.

    Cette classe surcharge la méthode `compute_loss` pour introduire une pondération
    des classes dans la fonction de perte (CrossEntropyLoss). Les poids sont calculés
    automatiquement à chaque appel pour contrer le déséquilibre des classes dans le
    dataset, donnant plus d'importance aux classes sous-représentées.
    """
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        """
        Calcule la perte en appliquant des poids de classe.

        Args:
            model (nn.Module): Le modèle en cours d'entraînement.
            inputs (dict): Un dictionnaire contenant les tenseurs d'entrée et les labels.
            return_outputs (bool): Si True, retourne aussi les sorties du modèle.

        Returns:
            torch.Tensor | tuple[torch.Tensor, dict]: La perte calculée, ou un tuple
                                                      contenant la perte et les sorties du modèle.
        """
        class_weights_np = compute_class_weight('balanced', classes=np.unique(labels), y=labels)
        class_weights = torch.tensor(class_weights_np, dtype=torch.float).to(model.device)

        labels_input = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get('logits')
        loss_fct = CrossEntropyLoss(weight=class_weights)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels_input.view(-1))
        return (loss, outputs) if return_outputs else loss

def compute_metrics_detailed(pred):
    """
    Calcule et retourne un dictionnaire de métriques pour l'évaluation.

    Cette fonction est conçue pour être utilisée avec l'API `Trainer` de Hugging Face.
    Elle calcule l'accuracy, les scores F1 (pondéré et macro), ainsi que le score
    F1 pour chaque classe individuelle.

    Args:
        pred (transformers.EvalPrediction): Un objet contenant les prédictions (logits)
                                            et les vrais labels (`label_ids`).

    Returns:
        dict: Un dictionnaire contenant les noms des métriques et leurs valeurs.
    """
    labels = pred.label_ids
    preds = pred.predictions.argmax(axis=1)
    f1_per_class = f1_score(labels, preds, average=None)
    class_perf = {f"f1_class_{le.classes_[i]}": f1 for i, f1 in enumerate(f1_per_class)}

    return {
        "accuracy": accuracy_score(labels, preds),
        "f1_weighted": f1_score(labels, preds, average="weighted"),
        "f1_macro": f1_score(labels, preds, average="macro"),
        **class_perf
    }

class DetailedLoggingCallback(TrainerCallback):
    """
    Callback pour afficher un résumé des performances après chaque évaluation.

    Ce callback se déclenche à la fin de chaque époque d'évaluation. Il affiche
    le score F1 pondéré, signale si un nouveau record a été établi, et liste
    les classes les plus faibles (celles avec un F1 < 0.5) pour faciliter le
    suivi et le débogage de l'entraînement.
    """
    def __init__(self):
        self.best_f1 = 0
    def on_evaluate(self, args, state, control, **kwargs):
        if state.epoch is not None:
            last_eval = next((log for log in reversed(state.log_history) if 'eval_f1_weighted' in log), None)
            if last_eval:
                current_f1 = last_eval.get('eval_f1_weighted', 0)
                is_best = current_f1 > self.best_f1
                if is_best: self.best_f1 = current_f1

                print(f"\n--- Fin Époque {int(state.epoch)} ---")
                print(f"  F1-weighted: {current_f1:.4f} {'⭐ NOUVEAU RECORD!' if is_best else ''}")
                weak_classes = [f"{key.split('_')[-1]}: {value:.3f}" for key, value in last_eval.items() if key.startswith('eval_f1_class_') and value < 0.5]
                if weak_classes: print(f"  Classes faibles: {', '.join(weak_classes[:3])}")

class PlotMetricsCallback(TrainerCallback):
    """
    Callback pour visualiser les courbes d'apprentissage à la fin de l'entraînement.

    Ce callback collecte les métriques (perte, F1, etc.) à chaque log et, une fois
    l'entraînement terminé, génère et affiche des graphiques montrant l'évolution
    de la perte (entraînement vs validation) et des performances sur l'ensemble de
    validation au fil des époques.
    """
    def __init__(self):
        self.history = []
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is not None and 'eval_loss' in logs:
            self.history.append(logs)
    def on_train_end(self, args, state, control, **kwargs):
        if not self.history: return
        df_log = pd.DataFrame(self.history)
        plt.figure(figsize=(14, 5))
        plt.subplot(1, 2, 1)
        plt.plot(df_log['epoch'], df_log['loss'], label='Train Loss')
        plt.plot(df_log['epoch'], df_log['eval_loss'], label='Validation Loss', color='orange')
        plt.title('Pertes (Loss)'); plt.xlabel('Époque'); plt.grid(True); plt.legend()
        plt.subplot(1, 2, 2)
        plt.plot(df_log['epoch'], df_log['eval_f1_weighted'], label='F1 Weighted', color='green')
        plt.plot(df_log['epoch'], df_log['eval_accuracy'], label='Accuracy', color='red')
        plt.title('Performances (Validation)'); plt.xlabel('Époque'); plt.grid(True); plt.legend()
        plt.tight_layout(); plt.show()

class RAMCleanupCallback(TrainerCallback):
    """
    Callback pour forcer le nettoyage de la mémoire CPU et GPU.

    À la fin de chaque époque, ce callback exécute le ramasse-miettes de Python (`gc.collect`)
    et vide le cache mémoire de la carte graphique (`torch.cuda.empty_cache`).
    C'est une mesure préventive pour éviter les erreurs "Out of Memory" lors
    d'entraînements longs avec de gros modèles.
    """
    def on_epoch_end(self, args, state, control, **kwargs):
        gc.collect()
        torch.cuda.empty_cache()

In [ ]:
# ====== 6. PROCESSUS D'ENTRAÎNEMENT PAR MODÈLE ======

# Arguments d'entraînement communs à tous les modèles
TRAINING_ARGS_COMMON = {
    "eval_strategy": "epoch",
    "save_strategy": "epoch",
    "learning_rate": 1e-5,
    "num_train_epochs": 50,
    "weight_decay": 0.1,
    "warmup_steps": 500,
    "lr_scheduler_type": "cosine_with_restarts",
    "logging_strategy": "epoch",
    "load_best_model_at_end": True,
    "metric_for_best_model": "f1_weighted",
    "greater_is_better": True,
    "fp16": True,
    "report_to": "none",
    "save_total_limit": 2,
}

# Dictionnaire contenant les configurations pour chaque modèle à entraîner.
model_configs = {
    "camembert": {
        "tokenizer": CamembertTokenizer.from_pretrained("camembert-base"),
        "model_class": CamembertForSequenceClassification,
        "pretrained_name": "camembert-base",
        "training_args": TrainingArguments(
            output_dir="./results_cam", per_device_train_batch_size=64, per_device_eval_batch_size=96, **TRAINING_ARGS_COMMON),
        "model_config": None
    },
    "flaubert": {
        "tokenizer": FlaubertTokenizer.from_pretrained("flaubert/flaubert_base_cased"),
        "model_class": FlaubertForSequenceClassification,
        "pretrained_name": "flaubert/flaubert_base_cased",
        "training_args": TrainingArguments(
            output_dir="./results_fla", per_device_train_batch_size=48, per_device_eval_batch_size=96, **TRAINING_ARGS_COMMON),
        "model_config": FlaubertConfig.from_pretrained("flaubert/flaubert_base_cased", num_labels=len(le.classes_), dropout=0.3, attention_dropout=0.3)
    }
}

# Boucle principale d'entraînement
for model_key, config in model_configs.items():
    print("\n" + "="*50)
    print(f" DÉBUT DE L'ENTRAÎNEMENT : {model_key.upper()} ")
    print("="*50)

    # --- 1. Préparation des datasets spécifiques au tokenizer ---
    tokenizer = config["tokenizer"]
    full_ds = TextDS(texts, labels, tokenizer)
    train_ds = Subset(full_ds, train_idx)
    val_ds = Subset(full_ds, val_idx)

    # --- 2. Initialisation du modèle ---
    model_params = {"pretrained_model_name_or_path": config["pretrained_name"]}
    if config["model_config"]:
        model_params["config"] = config["model_config"]
    else:
        model_params["num_labels"] = len(le.classes_)

    model = config["model_class"].from_pretrained(**model_params).to(device)

    # --- 3. Initialisation du Trainer ---
    plot_metrics_cb = PlotMetricsCallback()
    trainer = WeightedTrainer(
        model=model,
        args=config["training_args"],
        train_dataset=train_ds,
        eval_dataset=val_ds,
        data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
        compute_metrics=compute_metrics_detailed,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=8), DetailedLoggingCallback(), plot_metrics_cb, RAMCleanupCallback()]
    )

    # --- 4. Entraînement ---
    trainer.train()

    # --- 5. Sauvegarde des artefacts ---
    print(f"\n--- Sauvegarde des artefacts pour {model_key} ---")
    best_checkpoint = trainer.state.best_model_checkpoint
    final_model = config["model_class"].from_pretrained(best_checkpoint).to(device)

    # Sauvegarde du modèle sur Drive
    model_save_path = os.path.join(BASE_PATH, f"{model_key}2_model")
    final_model.save_pretrained(model_save_path)
    print(f"Modèle sauvegardé dans : {model_save_path}")

    # Sauvegarde des logits sur Drive
    logits = trainer.predict(full_ds).predictions
    logits_save_path = os.path.join(BASE_PATH, f"{model_key}2_logits.pt")
    torch.save(torch.tensor(logits), logits_save_path)
    print(f"Logits sauvegardés dans : {logits_save_path}")

    # --- 6. Rapport final ---
    print("\n--- Rapport de classification final sur la validation ---")
    predictions = trainer.predict(val_ds)
    preds = predictions.predictions.argmax(axis=1)
    print(classification_report(predictions.label_ids, preds, target_names=le.classes_))

    # --- 7. Nettoyage ---
    del model, trainer, final_model, full_ds, train_ds, val_ds, plot_metrics_cb
    gc.collect()
    torch.cuda.empty_cache()

# Sauvegarde des étiquettes une seule fois
torch.save(torch.tensor(labels), os.path.join(BASE_PATH, "true_labels_final.pt"))
print("\n\n✅ Processus d'entraînement pour tous les modèles terminé.")

PARTIE 2 : Fusion et Stacking des Modèles

In [ ]:
# ==============================================================================
# NOTEBOOK DE FUSION ET STACKING POUR MODÈLES TEXTE (AVANCÉ)
#
# Objectif : Combiner les prédictions (logits) de plusieurs modèles de base
# pour améliorer les performances.
#
#   1. Préparation des Données : Chargement des logits de 3 modèles de base
#      (CamemBERT, Flaubert, et un modèle booster externe).
#
#   2. Fusion Pondérée : Optimisation des poids pour une moyenne pondérée
#      des probabilités des modèles de base.
#
#   3. Stacking :
#      a. Préparation des features pour les méta-modèles en utilisant une
#         fusion pondérée fixe des logits de base.
#      b. Entraînement de deux méta-modèles (LightGBM et un MLP) avec
#         recherche d'hyperparamètres via Optuna.
#      c. Fusion des prédictions des méta-modèles.
#      d. Ajustement fin des seuils de classification pour maximiser le F1-Score.
#
#   4. Sauvegarde : Le méta-modèle final, ses poids de fusion et les
#      seuils optimaux sont sauvegardés.
# ==============================================================================

In [ ]:
# Installation des bibliothèques supplémentaires nécessaires pour cette partie
!pip install optuna lightgbm -q

In [ ]:
# Montage de Google Drive pour accéder aux fichiers depuis Colab
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# ====== 1. BIBLIOTHEQUES ======

# Outils système et gestion de fichiers
import os
import json
import shutil
import joblib
import itertools

# Manipulation de données et calcul scientifique
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.special import softmax

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

# Outils de modélisation et d'évaluation
import lightgbm as lgb
import optuna
from sklearn.metrics import f1_score, classification_report
from sklearn.model_selection import StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight

# Configuration de l'affichage pour Optuna pour un meilleur suivi
optuna.logging.set_verbosity(optuna.logging.INFO)

In [ ]:
# ====== 2. MISE EN PLACE DE L'ESPACE DE TRAVAIL ======

# Définition des chemins
# Le BASE_PATH est hérité de la PARTIE 1
DRIVE_PATH = BASE_PATH
LOCAL_PATH = "/content/data_stacking" # Dossier local dédié pour cette partie
os.makedirs(LOCAL_PATH, exist_ok=True)

# Création des dossiers de sortie sur le Drive
META_MODEL_DIR = os.path.join(DRIVE_PATH, "meta_modeles_texte")
os.makedirs(META_MODEL_DIR, exist_ok=True)


print("--- Copie des artefacts depuis Google Drive vers l'espace local ---")

# Liste des fichiers requis : logits des modèles de base et fichiers de configuration
files_to_copy = [
    "camembert2_logits.pt",
    "flaubert2_logits.pt",
    "booster_texte_logits.pt", # Ajout du troisième modèle
    "true_labels_final.pt",
    "val_indices.json",
    "label_encoder_final.pkl" # Utilisé pour les noms des classes
]

for filename in files_to_copy:
    src = os.path.join(DRIVE_PATH, filename)
    dst = os.path.join(LOCAL_PATH, filename)
    if os.path.exists(src):
        shutil.copy(src, dst)
        print(f"✅ {filename} copié.")
    else:
        # Affiche un avertissement si un fichier est manquant
        print(f"❌ AVERTISSEMENT : Le fichier {filename} est manquant sur le Drive.")

In [ ]:
# ====== 3. CHARGEMENT ET PRÉPARATION DES DONNÉES ======

# Chargement des logits des modèles de base
# Les logits sont immédiatement convertis en numpy arrays sur CPU
all_logits = {
    "camembert": torch.load(os.path.join(LOCAL_PATH, "camembert2_logits.pt")).cpu().numpy(),
    "flaubert": torch.load(os.path.join(LOCAL_PATH, "flaubert2_logits.pt")).cpu().numpy(),
    "booster_texte": torch.load(os.path.join(LOCAL_PATH, "booster_texte_logits.pt")).cpu().numpy()
}
print(f"Logits chargés pour les modèles : {list(all_logits.keys())}")

# Chargement des étiquettes, indices de validation, et noms des classes
labels_full = torch.load(os.path.join(LOCAL_PATH, "true_labels_final.pt")).numpy()
le = joblib.load(os.path.join(LOCAL_PATH, 'label_encoder_final.pkl'))
class_names = le.classes_

with open(os.path.join(LOCAL_PATH, "val_indices.json"), "r") as f:
    val_idx = np.array(json.load(f))

# Séparation des données pour l'entraînement et la validation des méta-modèles
train_idx = np.setdiff1d(np.arange(len(labels_full)), val_idx)
y_train, y_val = labels_full[train_idx], labels_full[val_idx]

# Application de la fonction softmax pour convertir les logits en probabilités
# Certains modèles peuvent déjà retourner des probabilités, on vérifie avant d'appliquer.
probs_train = {}
probs_val = {}
for name, log in all_logits.items():
    # Applique softmax si les lignes ne somment pas déjà à 1 (avec une tolérance)
    if (np.abs(np.sum(log, axis=1) - 1) > 1e-4).any():
        print(f"Application de Softmax sur les logits de '{name}'...")
        log = softmax(log, axis=1)
    probs_train[name] = log[train_idx]
    probs_val[name] = log[val_idx]


print(f"\nDonnées prêtes pour le méta-apprentissage :")
print(f"  - {len(y_train)} échantillons d'entraînement")
print(f"  - {len(y_val)} échantillons de validation")
print(f"  - Shape des probabilités (ex: camembert) : {probs_train['camembert'].shape}")

In [ ]:
# ====== 4. FONCTIONS UTILES POUR LE MÉTA-APPRENTISSAGE ======

# --- Fonctions de Fusion ---

def weighted_fusion(probas_list, weights):
    """
    Fusionne des listes de probabilités en utilisant une pondération.
    Utilise np.tensordot pour un calcul efficace.

    Args:
        probas_list (list[np.ndarray]): Liste des tableaux de probabilités.
        weights (np.ndarray): Tableau des poids correspondants.

    Returns:
        np.ndarray: Tableau des probabilités fusionnées.
    """
    stacked_probas = np.stack(probas_list, axis=0)
    return np.tensordot(weights, stacked_probas, axes=(0, 0))

def objective_fusion(trial, probas_dict, y_true):
    """
    Fonction objective pour Optuna visant à trouver les meilleurs poids de fusion.

    Cette fonction suggère un poids pour chaque modèle, normalise ces poids pour
    que leur somme soit égale à 1, puis calcule le F1-score pondéré qui en
    résulte. Ce score est retourné pour être maximisé par Optuna.

    Args:
        trial (optuna.trial.Trial): L'objet d'essai d'Optuna qui gère les
            hyperparamètres.
        probas_dict (dict): Dictionnaire où les clés sont les noms des modèles
            et les valeurs leurs probabilités de prédiction (np.ndarray).
        y_true (np.ndarray): Le tableau des véritables labels.

    Returns:
        float: Le score F1 pondéré pour la combinaison de poids actuelle.
    """
    weights = {name: trial.suggest_float(name, 0.0, 1.0) for name in probas_dict.keys()}
    s = sum(weights.values())
    if s < 1e-6: return 0.0 # Évite la division par zéro

    # Normalisation pour que la somme des poids soit égale à 1
    weights_normalized = {name: w / s for name, w in weights.items()}

    fused_probas = weighted_fusion(list(probas_dict.values()), np.array(list(weights_normalized.values())))
    preds = np.argmax(fused_probas, axis=1)

    return f1_score(y_true, preds, average="weighted")


# --- Fonctions d'Ajustement des Seuils ---

def apply_thresholds(probas, thresholds):
    """
    Applique des seuils de classification personnalisés pour chaque classe.

    Pour chaque échantillon, si aucune probabilité de classe ne dépasse son
    seuil respectif, la classe avec la probabilité la plus élevée est
    choisie par défaut. Sinon, la classe choisie est celle qui a la plus
    haute probabilité parmi celles qui ont passé leur seuil.

    Args:
        probas (np.ndarray): Un tableau 2D de probabilités (échantillons x classes).
        thresholds (np.ndarray): Un tableau 1D de seuils, un pour chaque classe.

    Returns:
        np.ndarray: Un tableau 1D contenant les prédictions finales (entiers).
    """
    n_samples, n_classes = probas.shape
    preds = np.zeros(n_samples, dtype=int)
    for i in range(n_samples):
        sample_probs = probas[i]
        passed_threshold = np.where(sample_probs >= thresholds)[0]
        if len(passed_threshold) == 0:
            preds[i] = np.argmax(sample_probs)
        else:
            # Parmi les classes ayant passé le seuil, on choisit celle avec la plus haute probabilité
            preds[i] = passed_threshold[np.argmax(sample_probs[passed_threshold])]
    return preds

def threshold_tuning(y_true, probas):
    """
    Optimise les seuils de classification pour maximiser le F1-score.

    Cette fonction parcourt chaque classe une par une et teste une grille de
    valeurs de seuil possibles. Elle conserve le seuil qui maximise le
    F1-score pondéré global avant de passer à la classe suivante.

    Args:
        y_true (np.ndarray): Le tableau des véritables labels.
        probas (np.ndarray): Le tableau des probabilités prédites par le modèle.

    Returns:
        np.ndarray: Le tableau des seuils optimisés.
    """
    n_classes = probas.shape[1]
    best_thresholds = np.full(n_classes, 0.5) # Commence avec des seuils par défaut
    best_f1 = f1_score(y_true, apply_thresholds(probas, best_thresholds), average='weighted')

    print("Début de l'optimisation des seuils...")
    for c in range(n_classes):
        threshold_candidates = np.linspace(0.1, 0.9, 17) # Grille de recherche
        for thr in threshold_candidates:
            current_thresholds = best_thresholds.copy()
            current_thresholds[c] = thr
            preds = apply_thresholds(probas, current_thresholds)
            f1 = f1_score(y_true, preds, average='weighted')
            if f1 > best_f1:
                best_f1 = f1
                best_thresholds = current_thresholds.copy()

    print(f"Optimisation terminée. F1-score après ajustement des seuils : {best_f1:.5f}")
    return best_thresholds


# --- Classes et Fonctions pour les Méta-Modèles ---

class MetaMLP(nn.Module):
    """
    Méta-modèle de type Perceptron Multi-Couches (MLP) simple.

    Cette classe définit un réseau de neurones simple avec une couche cachée,
    une fonction d'activation ReLU et du dropout, destiné à être utilisé
    comme méta-classifieur dans un ensemble de stacking.

    Args:
        input_dim (int): La dimensionnalité des features d'entrée.
        hidden_dim (int): Le nombre de neurones dans la couche cachée.
        n_classes (int): Le nombre de classes de sortie.
        dropout_rate (float, optional): Le taux de dropout. Défaut à 0.3.
    """
    def __init__(self, input_dim, hidden_dim, n_classes, dropout_rate=0.3):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim, n_classes)
        )

    def forward(self, x):
        """
        Définit la passe avant du modèle.

        Args:
            x (torch.Tensor): Le tenseur d'entrée.

        Returns:
            torch.Tensor: Les logits de sortie du modèle.
        """
        return self.net(x)

def train_meta_model(trial, model_type, X_train, y_train, X_val, y_val):
    """
    Entraîne un méta-modèle (LGBM ou MLP) avec des hyperparamètres d'Optuna.

    Cette fonction gère le processus d'entraînement pour un méta-modèle donné.
    Elle utilise l'objet `trial` d'Optuna pour suggérer des hyperparamètres,
    entraîne le modèle, l'évalue sur l'ensemble de validation et retourne
    le modèle entraîné ainsi que ses performances.

    Args:
        trial (optuna.trial.Trial): L'objet d'essai d'Optuna.
        model_type (str): Le type de modèle à entraîner ('lgbm' or 'mlp').
        X_train (np.ndarray): Les données d'entraînement (features).
        y_train (np.ndarray): Les étiquettes d'entraînement.
        X_val (np.ndarray): Les données de validation (features).
        y_val (np.ndarray): Les étiquettes de validation.

    Returns:
        tuple: Un tuple contenant :
            - model (object): L'instance du modèle entraîné.
            - val_probas (np.ndarray): Les probabilités prédites sur
              l'ensemble de validation.
            - f1 (float): Le score F1 pondéré sur l'ensemble de validation.
    """
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    num_classes = len(class_names)

    # Calcul des poids de classe pour gérer le déséquilibre
    class_weights_np = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)

    if model_type == 'lgbm':
        params = {
            'objective': 'multiclass', 'num_class': num_classes, 'metric': 'multi_logloss',
            'learning_rate': trial.suggest_float('lr', 1e-3, 0.1, log=True),
            'num_leaves': trial.suggest_int('num_leaves', 20, 50),
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'n_estimators': 1000, 'random_state': 42, 'n_jobs': -1
        }
        model = lgb.LGBMClassifier(**params)
        sample_weights = np.array([class_weights_np[np.where(np.unique(y_train) == y)[0][0]] for y in y_train])
        model.fit(X_train, y_train, sample_weight=sample_weights,
                  eval_set=[(X_val, y_val)],
                  callbacks=[lgb.early_stopping(10, verbose=False)])

        val_probas = model.predict_proba(X_val)

    elif model_type == 'mlp':
        input_dim = X_train.shape[1]
        hidden_dim = trial.suggest_int('hidden_dim', 128, 512)
        lr = trial.suggest_float('lr', 1e-4, 1e-2, log=True)

        class_weights_tensor = torch.tensor(class_weights_np, dtype=torch.float32).to(device)
        model = MetaMLP(input_dim, hidden_dim, num_classes).to(device)
        optimizer = optim.Adam(model.parameters(), lr=lr)
        criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)

        train_ds = TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.long))
        train_loader = DataLoader(train_ds, batch_size=256, shuffle=True)

        model.train()
        for epoch in range(15): # Nombre fixe d'époques pour l'entraînement
            for xb, yb in train_loader:
                xb, yb = xb.to(device), yb.to(device)
                optimizer.zero_grad()
                logits = model(xb)
                loss = criterion(logits, yb)
                loss.backward()
                optimizer.step()

        model.eval()
        with torch.no_grad():
            val_probas = model(torch.tensor(X_val, dtype=torch.float32).to(device)).cpu().numpy()
            val_probas = softmax(val_probas, axis=1)

    # Évaluation du F1-score pour Optuna
    preds = np.argmax(val_probas, axis=1)
    f1 = f1_score(y_val, preds, average='weighted')

    return model, val_probas, f1

In [ ]:
# ====== 5. STACKING : PRÉPARATION DES FEATURES ET ENTRAÎNEMENT DES MÉTA-MODÈLES ======

# --- 5.1 Préparation des Données de Stacking ---
# Les features pour nos méta-modèles sont créées en fusionnant les probabilités
# des modèles de base. Nous utilisons ici une pondération fixe déterminée
# empiriquement ou par une analyse antérieure.

# Poids fixes pour la fusion initiale créant les features du stacking
fixed_weights_stacking = np.array([0.978, 0.022, 0.299])
fixed_weights_stacking /= fixed_weights_stacking.sum() # Normalisation

X_train_stack = weighted_fusion(list(probs_train.values()), fixed_weights_stacking)
X_val_stack = weighted_fusion(list(probs_val.values()), fixed_weights_stacking)

print(f"Shape des features de Stacking (Train): {X_train_stack.shape}")

# --- 5.2 Entraînement et Optimisation des Méta-Modèles ---
meta_models_trained = {}
meta_probas_val = {}

# Itération sur les types de méta-modèles à entraîner
for model_name in ['lgbm', 'mlp']:
    print(f"\n--- Optimisation des hyperparamètres pour le méta-modèle : {model_name.upper()} ---")

    # Création de l'étude Optuna pour maximiser le F1-score
    study = optuna.create_study(direction="maximize")

    # Définition de la fonction objective pour l'étude
    objective_func = lambda trial: train_meta_model(trial, model_name, X_train_stack, y_train, X_val_stack, y_val)[2]

    study.optimize(objective_func, n_trials=30, n_jobs=-1) # 30 essais pour trouver les meilleurs hyperparamètres

    print(f"Meilleurs hyperparamètres pour {model_name}: {study.best_params}")
    print(f"Meilleur F1-Score obtenu lors de l'optimisation : {study.best_value:.5f}")

    # Entraînement du modèle final avec les meilleurs hyperparamètres trouvés
    final_model, final_probas, _ = train_meta_model(study.best_trial, model_name, X_train_stack, y_train, X_val_stack, y_val)

    meta_models_trained[model_name] = final_model
    meta_probas_val[model_name] = final_probas

In [ ]:
# ====== 6. FUSION FINALE, AJUSTEMENT DES SEUILS ET SAUVEGARDE ======

print("\n--- Recherche des poids de fusion optimaux pour les méta-modèles ---")
study_fusion_meta = optuna.create_study(direction="maximize")
study_fusion_meta.optimize(
    lambda trial: objective_fusion(trial, meta_probas_val, y_val),
    n_trials=100
)

best_weights_meta = {name: w for name, w in study_fusion_meta.best_params.items()}
s = sum(best_weights_meta.values())
best_weights_meta_normalized = {name: w / s for name, w in best_weights_meta.items()}

print(f"\nMeilleur F1-Score (Fusion des méta-modèles) : {study_fusion_meta.best_value:.5f}")
print("Poids optimaux pour la fusion finale :", best_weights_meta_normalized)

# Fusion finale des probabilités des méta-modèles
final_probas = weighted_fusion(
    list(meta_probas_val.values()),
    np.array(list(best_weights_meta_normalized.values()))
)

# Ajustement final des seuils sur les probabilités fusionnées
final_thresholds = threshold_tuning(y_val, final_probas)

# --- Sauvegarde des artefacts finaux ---
print("\n--- Sauvegarde des artefacts du méta-modèle ---")
# Sauvegarde des modèles entraînés
for name, model in meta_models_trained.items():
    path = os.path.join(META_MODEL_DIR, f"meta_model_{name}.pkl")
    if name == 'mlp':
        path = os.path.join(META_MODEL_DIR, f"meta_model_{name}.pth")
        torch.save(model.state_dict(), path)
    else:
        joblib.dump(model, path)
    print(f"Méta-modèle {name} sauvegardé dans : {path}")

# Sauvegarde des poids de fusion et des seuils
np.save(os.path.join(META_MODEL_DIR, "final_fusion_weights.npy"), list(best_weights_meta_normalized.values()))
np.save(os.path.join(META_MODEL_DIR, "final_thresholds.npy"), final_thresholds)
print("Poids de fusion et seuils sauvegardés.")

In [ ]:
# ====== 7. RÉSULTATS FINAUX ======

print("\n" + "="*60)
print("              RÉSULTATS FINAUX SUR L'ENSEMBLE DE VALIDATION")
print("="*60)

# Application des seuils optimisés pour obtenir les prédictions finales
final_preds = apply_thresholds(final_probas, final_thresholds)

print("\n--- Rapport de Classification Final (Stacking + Fusion + Seuils) ---")
print(classification_report(y_val, final_preds, target_names=class_names, digits=4))

print("\n✅ Processus de méta-apprentissage terminé.")

In [ ]:
!pip freeze > "/content/drive/MyDrive/Colab Notebooks/requirements_Texte.txt"